# Exact cover

Exact cover algorithm implemented by QUBO in quantum annealer

Exact cover problem (https://en.wikipedia.org/wiki/Exact_cover): we have set of elements $v \in U$ and set of subset $V_i \in V$ of the elements $v$. How to select some of the subsets so that each element of the set is in exactly one of the subsets. So selected subsets cover exactly the entire set. Exact cover problem belongs to class NP-complete. Knuth's Algorithm X is one classical quite efficient algorithm to solve Exact cover.

There appears to be very straightforward algorithm implementation in QUBO, where we need only $|V|$. Each row and column in QUBO represents subset so the size of QUBO is $|V| \times |V|$. Weight of diagonal cell is $-|V_i|$. Weight of other cells is 0 if two subsets $V_i$ and $V_j$ do not have any common elements. Otherwise weight of the non-diagonal cells is $|V_i|+|V_j|+2$, so that if both these subset are selected, then penalty will be $+2$.

To get the exact cover, algorithm should found the energy state of $-|U|$.

This way QUBO implements following constraints:

- total number of elements in subsets should be as high as possible, our target is of course $-|U|$.
- each element appears only in one of the subsets.

In [1]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector

## Define sets

In [13]:
U = [0,1,2,3,4,5,6,7,8,9]
V = [[0,1,2], [2,3], [6,7,8,9], [7,9], [4,5,6], [4,5], [9]]

edge_names = {}
for i,v in enumerate(V):
    edge_names[i] = str(v)

def result_info(sampleset):
    r = sampleset.first.sample
    r = [k for k, v in r.items() if v==1]
    print('Edges: ' + str(r))

## Create QUBO

Constraints
- total number of elements in subset should be $|U|$
- each element only in one subset

In [14]:
Q = np.zeros((len(V),len(V)))

# Total elements constraint

max_count = 0

for i in range(len(V)):
    Q[i][i] =- len(V[i])
    max_count += len(V[i])

# each element only in one subset

for a in U:
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            if a in V[i] and a in V[j]:
                Q[i][j] = len(V[i]) + len(V[j]) + 2
print(Q)

[[-3.  7.  0.  0.  0.  0.  0.]
 [ 0. -2.  0.  0.  0.  0.  0.]
 [ 0.  0. -4.  8.  9.  0.  7.]
 [ 0.  0.  0. -2.  0.  0.  5.]
 [ 0.  0.  0.  0. -3.  7.  0.]
 [ 0.  0.  0.  0.  0. -2.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.]]


## Creat BQM from QUBO

In [15]:
bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
bqm = bqm.relabel_variables(edge_names, inplace=False)

## Local deterministic solver

In [16]:
sampleset = dimod.ExactSolver().sample(bqm)
print(sampleset.truncate(10))

  [0, 1, 2] [2, 3] [4, 5, 6] [4, 5] [6, 7, 8, 9] [7, 9] [9] energy num_oc.
0         1      0         0      1            1      0   0   -9.0       1
1         0      1         0      1            1      0   0   -8.0       1
2         1      0         1      0            0      1   0   -8.0       1
3         1      0         0      1            0      1   0   -7.0       1
4         1      0         1      0            0      0   1   -7.0       1
5         1      0         0      0            1      0   0   -7.0       1
6         0      1         1      0            0      1   0   -7.0       1
7         0      0         0      1            1      0   0   -6.0       1
8         0      1         0      1            0      1   0   -6.0       1
9         0      1         0      0            1      0   0   -6.0       1
['BINARY', 10 rows, 10 samples, 7 variables]


In [17]:
result_info(sampleset)

Edges: ['[0, 1, 2]', '[4, 5]', '[6, 7, 8, 9]']


## Local heuristic classical solver

In [18]:
t1 = time.time()
sampleset2 = SimulatedAnnealingSampler().sample(bqm, num_reads=500)
t2 = time.time()
print('Time used by solver (s): {:.1f}'.format((t2-t1)))
print(sampleset2.aggregate().truncate(10))

Time used by solver (s): 0.1
  [0, 1, 2] [2, 3] [4, 5, 6] [4, 5] [6, 7, 8, 9] [7, 9] [9] energy num_oc.
0         1      0         0      1            1      0   0   -9.0     313
1         1      0         1      0            0      1   0   -8.0     118
2         0      1         0      1            1      0   0   -8.0      37
3         0      1         1      0            0      1   0   -7.0      21
4         1      0         0      1            0      1   0   -7.0       6
5         1      0         1      0            0      0   1   -7.0       2
6         0      1         0      1            0      1   0   -6.0       2
7         0      1         1      0            0      0   1   -6.0       1
['BINARY', 8 rows, 500 samples, 7 variables]


In [19]:
result_info(sampleset2)

Edges: ['[0, 1, 2]', '[4, 5]', '[6, 7, 8, 9]']


## Quantum solver

In [20]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [21]:
sampleset3 = EmbeddingComposite(machine).sample(bqm, num_reads=500)
print(sampleset3) 

  [0, 1, 2] [2, 3] [4, 5, 6] [4, 5] [6, 7, 8, 9] [7, 9] [9] energy num_oc. ...
0         1      0         0      1            1      0   0   -9.0     335 ...
1         0      1         0      1            1      0   0   -8.0      47 ...
2         1      0         1      0            0      1   0   -8.0      69 ...
3         1      0         0      1            0      1   0   -7.0       9 ...
4         0      1         1      0            0      1   0   -7.0       8 ...
5         1      0         1      0            0      0   1   -7.0      25 ...
6         1      0         0      0            1      0   0   -7.0       1 ...
7         0      1         0      1            0      1   0   -6.0       1 ...
8         0      1         1      0            0      0   1   -6.0       4 ...
9         1      0         0      1            0      0   1   -6.0       1 ...
['BINARY', 10 rows, 500 samples, 7 variables]


In [22]:
time = sampleset3.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset3.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(time))
print('Physical qubits used: {}'.format(qubits))

QPU time used (ms): 67.9
Physical qubits used: 7


In [23]:
result_info(sampleset3)

Edges: ['[0, 1, 2]', '[4, 5]', '[6, 7, 8, 9]']
